<a href="https://colab.research.google.com/github/caio-olubini/Case---Especialista-de-Dados-Ecomm-/blob/main/Parte%202%20-%20An%C3%A1lise%20de%20Sentimento/Sentiment_Analysis_Products_Reviews_PT_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando pacotes

In [ ]:
import pandas as pd
import time
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
try:
  from unidecode import unidecode
except ModuleNotFoundError:
  !pip install unidecode
  from unidecode import unidecode
from nltk.corpus import stopwords
import nltk
from scipy.stats import chi2_contingency

# Lendo datasets

In [ ]:
df_reviews = pd.read_csv('reviews.csv')
df_reviews.dropna(subset=['MSG_AVALIACAO'], inplace=True)
df_reviews['DT_HR_CRIACAO'] = pd.to_datetime(df_reviews['DT_HR_CRIACAO'], format='mixed')
df_reviews['COD_PEDIDO'] = df_reviews['COD_PEDIDO'].astype('Int64')

In [ ]:
df_sales = pd.read_csv('vendas.csv')
df_sales['DT_VENDA'] = pd.to_datetime(df_sales['DT_VENDA'])

# Análise exploratória

## Reviews

In [ ]:
df_reviews.sample(5)

In [ ]:
# Extract month from the 'DT_HR_CRIACAO' column
df_reviews['month'] = df_reviews['DT_HR_CRIACAO'].dt.to_period('M')

# Group by month and count distinct COD_AVALIACAO
reviews_per_month = df_reviews.groupby('month')['COD_AVALIACAO'].nunique()

# Create the plot
plt.figure(figsize=(8, 3))
ax = sns.barplot(x=reviews_per_month.index.astype(str), y=reviews_per_month.values)
plt.title('Reviews por mês', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Reviews', fontsize=12)
plt.xticks(rotation=0)

sns.despine()
plt.tight_layout()

plt.show()

### Produtos

In [ ]:
df_reviews['COD_PRODUTO'].nunique()

In [ ]:
df_reviews[['COD_PRODUTO', 'DES_PRODUTO']].value_counts(normalize=True)

O dataset contém reviews de apenas 3 produtos

Aparentemente temos uma atribuição errada da descrição **Cuide-se Bem Doçura na Pessegura o Boticário - Creme Hidratante Depilatório Corporal 150ml** para o código 333

### Mensagem de avaliação

In [ ]:
df_reviews['MSG_AVALIACAO'].nunique()

In [ ]:
df_reviews['MSG_AVALIACAO'].value_counts()[:10]

In [ ]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

# Function to preprocess text
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Normalize special characters
    text = unidecode(text)
    # Remove stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text

# Combine all reviews into a single string and preprocess
text = " ".join(preprocess_text(review) for review in df_reviews['MSG_AVALIACAO'])

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Notamos que temos alguns reviews "repetidos" e mais comuns, majoriatariamente relacionados a mensagens curtas e positivas como "muito bom" ou "amei".

Analisando nossa nuvem de palavras notamos que a maioria dos comentários está relacionada a caracteristicas inerentes a categoria de perfumaria (cheiro, frangrancia,fixação) associadas a palavras positivas (maravilhoso, bom, perfeito)

### Flag presente

In [ ]:
df_reviews['FLG_PRESENTE'].value_counts(normalize = True)

In [ ]:
df_reviews.groupby('COD_PRODUTO')['FLG_PRESENTE'].value_counts(normalize = True)

A maior parte dos comentários é referente a itens não presenteados.

O percentual presenteado do item 333 é consideravelmente menor dos que os demais, o que é esperado já que se trata de um hidratante.

### Estado (UF)

In [ ]:
df_reviews['ESTADO_AVALIADOR'].nunique()

In [ ]:
df_reviews['ESTADO_AVALIADOR'].value_counts(normalize=True)

### Duplicicades

In [ ]:
df_reviews[df_reviews.drop(columns = ['DT_HR_CRIACAO']).duplicated(keep=False)]

**802 reviews duplicados** encontrados na base, provavelmente resultado de problemas no processo de coleta.

### Valores faltantes

In [ ]:
df_reviews.isna().sum()

In [ ]:
# Extract month from the 'DT_HR_CRIACAO' column
df_reviews['month'] = df_reviews['DT_HR_CRIACAO'].dt.to_period('M')

# Calculate missing and total values per month
missing_per_month = df_reviews[df_reviews['COD_PEDIDO'].isna()].groupby('month').size()
total_per_month = df_reviews.groupby('month').size()

# Calculate the percentage of missing values
percentage_missing = (missing_per_month / total_per_month) * 100

# Plot the results
plt.figure(figsize=(8, 3))
ax = percentage_missing.plot(kind='bar')
plt.title('% Valores faltantes COD_PEDIDO X Mês', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Valores Faltantes (%)', fontsize=12)
plt.xticks(rotation=0)

# Remove top and right spines
sns.despine()

plt.ylim(0, 70)

**Valores faltantes:**

* COD_PEDIDO - aproximadamente metade dos registros não possuem o campo preenchido, o que deve afetar o cruzando com a base de vendas. O problema é um pouco mais acentuado no mês de Abril.

* ESTADO_AVALIADOR - 656 valores faltantes

* FLG_PRESENTE - 70

Pela restriçã de tempo e por entender que eles não necessariamente afetam o objetivo principal da análise, por hora não trataremos valores faltantes.

In [ ]:
df_sales = pd.read_csv('vendas.csv')
df_sales

In [ ]:
df_sales['COD_PEDIDO'].nunique()

## Vendas

In [ ]:
df_sales.sample(5)

In [ ]:
df_sales[['DT_VENDA', 'VLR_RECEITA_CAPTADA']].describe()

In [ ]:
# Extract month from the 'DT_VENDA' column
df_sales['month'] = pd.to_datetime(df_sales['DT_VENDA']).dt.to_period('M')

# Group by month and count distinct COD_PEDIDO
sales_per_month = df_sales.groupby('month')['COD_PEDIDO'].nunique()

# Create the plot
plt.figure(figsize=(8, 3))
ax = sns.barplot(x=sales_per_month.index.astype(str), y=sales_per_month.values)
plt.title('Pedidos por Mês', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Pedidos', fontsize=12)
plt.xticks(rotation=0)

# Remove top and right spines
sns.despine()


plt.show()

### Canal de venda (APP, SITE, MKTP)

In [ ]:
df_sales['DES_CANAL_VENDA_FINAL'].value_counts(normalize=True)

In [ ]:
df_sales.groupby('DES_CANAL_VENDA_FINAL')['VLR_RECEITA_CAPTADA'].mean()

In [ ]:
# Calculate the percentage of each COD_MATERIAL within each DES_CANAL_VENDA_FINAL
share_by_channel = df_sales.groupby(['DES_CANAL_VENDA_FINAL', 'COD_MATERIAL']).size().unstack(fill_value=0)
share_by_channel = share_by_channel.apply(lambda x: x / x.sum(), axis=1)

# Plot the results as a stacked bar chart
share_by_channel.plot(kind='bar', stacked=True, figsize=(10, 4))

plt.title('Share Material Canal', fontsize=16)
plt.xlabel('Canal', fontsize=12)
plt.ylabel('Share', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='COD_MATERIAL', bbox_to_anchor=(1.05, 1), loc='upper left')
sns.despine()
plt.tight_layout()
plt.show()

O dataset possui 4 canais de venda:

App, Site, Mktp In, Mktp Out

Tendo app a maior representação de pedidos e Mktp Out o melhor ticket médio.

### Produtos

In [ ]:
df_sales['COD_MATERIAL'].nunique()

In [ ]:
df_sales[['COD_MATERIAL', 'DES_MATERIAL']].value_counts(normalize=True)

In [ ]:
df_sales.groupby('COD_MATERIAL')['VLR_RECEITA_CAPTADA'].describe()

In [ ]:
plt.figure(figsize=(12, 4))
sns.boxplot(x='COD_MATERIAL', y='VLR_RECEITA_CAPTADA', data=df_sales, palette='viridis')
plt.title('Receita por produto', fontsize=16)
plt.xlabel('cod_material', fontsize=12)
plt.ylabel('Receita', fontsize=12)
sns.despine()
plt.ylim(0, 4000)
plt.show()

Aqui temos os mesmos três produtos observados na base de reviews, porém, com shares invertidos, o que pode indicar que diferentes materiais tem maior ou menor propensão a serem avaliados.

Podemos notar que o mesmo produto pode possuir valores de receita associada bastante variados!!

Novamente, vemos também observamos erros de atribuição de cod_material.

### Receita

In [ ]:
plt.figure(figsize=(12, 4))
sns.histplot(df_sales['VLR_RECEITA_CAPTADA'], bins=15)
plt.title('Distribution de Receita', fontsize=16)
plt.xlabel('Receita', fontsize=12)
plt.ylabel('Frequencia', fontsize=12)
sns.despine()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
sns.boxplot(x='DES_CANAL_VENDA_FINAL', y='VLR_RECEITA_CAPTADA', data=df_sales, palette='viridis')
plt.title('Receita por canal', fontsize=16)
plt.xlabel('Canal', fontsize=12)
plt.ylabel('Receita', fontsize=12)
sns.despine()
plt.ylim(0, 4000)
plt.show()

75% das vendas tem valor de até 1000 reais.

App e Site tem distribuições de receita muito parecidas.

Mktp In tem uma tendência central um pouco mais elevada.

Mkt Out possui distribuição bastante distinta, com valores de receita mais elevados.

### Duplicidades

In [ ]:
df_sales[df_sales.duplicated(subset = ['COD_PEDIDO'], keep=False)]

In [ ]:
df_sales[df_sales.duplicated(subset = ['COD_PEDIDO', 'COD_MATERIAL'], keep=False)]

No dataset de vendas temos duplicidades a nível de cod_pedido e material, nesse caso optaremos por agrupar a nível de pedido e SKU, somando a receita captada.

### Valores faltantes

In [ ]:
df_sales.isna().sum()

# Pré-processamento e Engenharia de recursos

## Duplicidade

In [ ]:
rows_to_drop = df_reviews.drop(columns = ['DT_HR_CRIACAO']).duplicated(keep=False)
df_reviews = df_reviews[~rows_to_drop]
df_reviews

In [ ]:
df_sales = df_sales.groupby(['COD_PEDIDO', 'COD_MATERIAL', 'DES_CANAL_VENDA_FINAL']).agg({'VLR_RECEITA_CAPTADA': 'sum', 'DT_VENDA': 'min'}).reset_index()
df_sales

## Cruzamento de bases

In [ ]:
df_reviews_merged = pd.merge(df_reviews, df_sales[['COD_PEDIDO', 'COD_MATERIAL', 'DES_CANAL_VENDA_FINAL', 'VLR_RECEITA_CAPTADA', 'DT_VENDA']], how = 'left', left_on = ['COD_PEDIDO', 'COD_PRODUTO'], right_on = ['COD_PEDIDO', 'COD_MATERIAL']).drop(columns = ['COD_MATERIAL'])
df_reviews_merged.sample(5)

In [ ]:
df_reviews_merged.isna().mean()

Como somente 13% dos reviews tem correspondência na base de vendas, a partir desse ponto seguiremos a análise desconsiderando dados cruzados.

Trazer análises a partir desses dados agregaria viés aos nossos resultados, já que provavelmente existe um motivo sistemático para que certos comentários não tenham correspondência.


## Criação de recursos - Região

In [ ]:
# Define the mapping from state to region
region_map = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Create the new 'REGIAO_AVALIADOR' column
df_reviews['REGIAO_AVALIADOR'] = df_reviews['ESTADO_AVALIADOR'].map(region_map)

# Display the first few rows with the new column
df_reviews[['ESTADO_AVALIADOR', 'REGIAO_AVALIADOR']].sample(5)

In [ ]:
# Group by region and count distinct reviews
reviews_by_region = df_reviews.groupby('REGIAO_AVALIADOR')['COD_AVALIACAO'].nunique().sort_values(ascending=False)

# Create the plot
plt.figure(figsize=(10, 4))
ax = sns.barplot(x=reviews_by_region.index, y=reviews_by_region.values, palette='viridis')

# Add titles and labels
plt.title('Distinct Reviews per Region', fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Number of Distinct Reviews', fontsize=12)
plt.xticks(rotation=0)

# Clean up the plot
sns.despine()
plt.tight_layout()
plt.show()

# Modelagem

Nesta seção, realizaremos a análise de sentimentos dos dados. Para isso, utilizaremos a função pipeline da biblioteca **Hugging Face**, uma ferramenta de alto nível que simplifica todo o processo de modelagem.

O modelo escolhido é o **bertweet-pt-sentiment**, que foi treinado especificamente com um vocabulário em português e é otimizado para a tarefa de análise de sentimentos.

In [ ]:
print("Carregando o modelo pré-treinado 'pysentimiento/bertweet-pt-sentiment'...")
try:
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="pysentimiento/bertweet-pt-sentiment",
        device = 0
    )
    print("✅ Modelo carregado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o modelo: {e}")

In [ ]:
df_reviews = pd.read_csv('reviews_classificados.csv')
df_reviews['DT_HR_CRIACAO'] = pd.to_datetime(df_reviews['DT_HR_CRIACAO'], format='mixed')
df_reviews['COD_PEDIDO'] = df_reviews['COD_PEDIDO'].astype('Int64')

'''reviews = df_reviews['MSG_AVALIACAO'].tolist()

print(f"\nIniciando a classificação de {len(reviews)} textos em lote...")
start_time = time.time()
classification_results = sentiment_pipeline(reviews, truncation=True, batch_size=8)
end_time = time.time()
print("✅ Classificação em lote concluída!")

elapsed_time = end_time - start_time
print(f"Tempo de execução: {elapsed_time:.2f} segundos")

labels = [r['label'].capitalize() for r in classification_results]
df_reviews['labels'] = labels'''

# Tempo de execução: 716.51 segundos -- Sem GPU


In [ ]:
df_reviews.to_csv('reviews_classificados.csv', index = False)

## Fluxo em produção

## Avaliação do modelo

# Insights

# Análise de Sentimento


In [ ]:
df_reviews['labels'].value_counts(normalize = True)

In [ ]:
# Filter reviews by sentiment
positive_reviews = df_reviews[df_reviews['labels'] == 'Pos']['MSG_AVALIACAO']
negative_reviews = df_reviews[df_reviews['labels'] == 'Neg']['MSG_AVALIACAO']
neutral_reviews = df_reviews[df_reviews['labels'] == 'Neu']['MSG_AVALIACAO']

# Combine reviews into single strings
positive_text = " ".join(review for review in positive_reviews)
negative_text = " ".join(review for review in negative_reviews)
neutral_text = " ".join(review for review in neutral_reviews)

# Preprocess the text for each sentiment
positive_text = preprocess_text(positive_text)
negative_text = preprocess_text(negative_text)
neutral_text = preprocess_text(neutral_text)

# Generate word clouds
wordcloud_pos = WordCloud(width=800, height=400, background_color='white').generate(positive_text)
wordcloud_neg = WordCloud(width=800, height=400, background_color='white').generate(negative_text)
wordcloud_neu = WordCloud(width=800, height=400, background_color='white').generate(neutral_text)

# Display the word clouds
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.title('Sentimento Positivo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.title('Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(wordcloud_neu, interpolation='bilinear')
plt.title('Sentimento Neutro', fontsize=16)
plt.axis("off")

plt.tight_layout()
plt.show()

## Sentimento ao longo do tempo

In [ ]:
df_reviews.groupby(['month'])['labels'].value_counts(normalize = True)


In [ ]:
# Group by month and sentiment, then count occurrences
sentiment_over_time = df_reviews.groupby(pd.Grouper(key='DT_HR_CRIACAO', freq='W'))['labels'].value_counts(normalize = True).reset_index()

# Plot the sentiment trends over time
plt.figure(figsize=(12, 3))
sns.lineplot(data = sentiment_over_time, x = 'DT_HR_CRIACAO', y = 'proportion', hue = 'labels')

# Add titles and labels
plt.title('Evolução do Sentimento ao Longo do Tempo', fontsize=16)
plt.xlabel('Semana', fontsize=12)
plt.ylabel('Proporção de Reviews', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='Sentimento')
sns.despine()
plt.tight_layout()
plt.show()

## Sentimento por produto

In [ ]:
# Group by product and sentiment, then count occurrences
sentiment_by_product = df_reviews.groupby(['COD_PRODUTO', 'labels']).size().unstack(fill_value=0)

# Normalize to get percentages
sentiment_by_product_percentage = sentiment_by_product.div(sentiment_by_product.sum(axis=1), axis=0)

# Create a 100% stacked bar chart
ax = sentiment_by_product_percentage.plot(kind='bar', stacked=True, figsize=(8, 4))

# Add titles and labels
plt.title('Distribuição de Sentimento por Produto', fontsize=16)
plt.xlabel('Código do Produto', fontsize=12)
plt.ylabel('Proporção de Reviews', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='Sentimento')

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))

# Clean up the plot
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_reviews['COD_PRODUTO'], df_reviews['labels'])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-Square Test for COD_PRODUTO vs labels")
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")

# Interpret the results
alpha = 0.05
if p < alpha:
    print("\nConclusion: There is a statistically significant association between product and sentiment (p < 0.05).")
else:
    print("\nConclusion: There is no statistically significant association between product and sentiment (p >= 0.05).")

### Ao longo do tempo

In [ ]:
df_reviews.groupby(['COD_PRODUTO', 'month'])['labels'].value_counts(normalize = True)

In [ ]:
for produto in sorted(df_reviews['COD_PRODUTO'].unique()):

  product_reviews = df_reviews[df_reviews['COD_PRODUTO'] == produto]

  g = product_reviews.groupby(pd.Grouper(key='DT_HR_CRIACAO', freq='W'))['labels'].value_counts(normalize = True).reset_index()

  plt.figure(figsize=(12, 3))
  plt.title(f'Sentimento ao longo do tempo, produto {produto}', fontsize=16)
  plt.xlabel('Semana', fontsize=12)
  plt.ylabel('Proporção de sentimento', fontsize=12)
  sns.lineplot(data=g, x='DT_HR_CRIACAO', y='proportion', hue='labels', palette='viridis')

  sns.despine()
  plt.tight_layout()
  plt.show()

### Deepdive comentários negativos 333

In [ ]:
df_reviews[(df_reviews['COD_PRODUTO'] == 333) & (df_reviews['month'] == '2025-02') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO'].sample(10)

In [ ]:
df_reviews[(df_reviews['COD_PRODUTO'] == 333) & (df_reviews['month'] == '2025-04') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO'].sample(10)

In [ ]:
# Filter reviews by sentiment
neg_reviews_feb = df_reviews[(df_reviews['COD_PRODUTO'] == 333) & (df_reviews['month'] == '2025-02') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']
neg_reviews_mar = df_reviews[(df_reviews['COD_PRODUTO'] == 333) & (df_reviews['month'] == '2025-03') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']
neg_reviews_apr = df_reviews[(df_reviews['COD_PRODUTO'] == 333) & (df_reviews['month'] == '2025-03') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']

# Combine reviews into single strings

feb_text = " ".join(review for review in neg_reviews_feb)
mar_text = " ".join(review for review in neg_reviews_mar)
apr_text = " ".join(review for review in neg_reviews_apr)

# Preprocess the text for each sentiment
feb_text = preprocess_text(feb_text)
mar_text = preprocess_text(mar_text)
apr_text = preprocess_text(apr_text)

# Generate word clouds
wordcloud_feb = WordCloud(width=800, height=400, background_color='white').generate(feb_text)
wordcloud_mar = WordCloud(width=800, height=400, background_color='white').generate(mar_text)
wordcloud_apr = WordCloud(width=800, height=400, background_color='white').generate(apr_text)

# Display the word clouds
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(wordcloud_feb, interpolation='bilinear')
plt.title('Fevereiro - Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(wordcloud_mar, interpolation='bilinear')
plt.title('Março - Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(wordcloud_apr, interpolation='bilinear')
plt.title('Abril - Sentimento Neutro', fontsize=16)
plt.axis("off")

plt.tight_layout()
plt.show()

### Deepdive comentários negativos 222

In [ ]:
df_reviews[(df_reviews['COD_PRODUTO'] == 222) & (df_reviews['month'] == '2025-04') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO'].sample(10)

In [ ]:
# Filter reviews by sentiment
neg_reviews_feb = df_reviews[(df_reviews['COD_PRODUTO'] == 222) & (df_reviews['month'] == '2025-02') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']
neg_reviews_mar = df_reviews[(df_reviews['COD_PRODUTO'] == 222) & (df_reviews['month'] == '2025-03') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']
neg_reviews_apr = df_reviews[(df_reviews['COD_PRODUTO'] == 222) & (df_reviews['month'] == '2025-03') & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']

# Combine reviews into single strings

feb_text = " ".join(review for review in neg_reviews_feb)
mar_text = " ".join(review for review in neg_reviews_mar)
apr_text = " ".join(review for review in neg_reviews_apr)

# Preprocess the text for each sentiment
feb_text = preprocess_text(feb_text)
mar_text = preprocess_text(mar_text)
apr_text = preprocess_text(apr_text)

# Generate word clouds
wordcloud_feb = WordCloud(width=800, height=400, background_color='white').generate(feb_text)
wordcloud_mar = WordCloud(width=800, height=400, background_color='white').generate(mar_text)
wordcloud_apr = WordCloud(width=800, height=400, background_color='white').generate(apr_text)

# Display the word clouds
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(wordcloud_feb, interpolation='bilinear')
plt.title('Fevereiro - Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(wordcloud_mar, interpolation='bilinear')
plt.title('Março - Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(wordcloud_apr, interpolation='bilinear')
plt.title('Abril - Sentimento Neutro', fontsize=16)
plt.axis("off")

plt.tight_layout()
plt.show()

### Deepdive comentários negativos 111

In [ ]:
df_reviews[(df_reviews['COD_PRODUTO'] == 111) & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']

In [ ]:
# Filter reviews by sentiment
neg_reviews = df_reviews[(df_reviews['COD_PRODUTO'] == 222) & (df_reviews['labels'] == 'Neg')]['MSG_AVALIACAO']


# Combine reviews into single strings
text = " ".join(review for review in neg_reviews)

# Preprocess the text for each sentiment
text = preprocess_text(text)

# Generate word clouds
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word clouds
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Sentimento Negativo', fontsize=16)
plt.axis("off")

plt.tight_layout()
plt.show()

## Sentiment by region


In [ ]:
# Group by region and sentiment, then count occurrences
sentiment_by_region = df_reviews.groupby(['REGIAO_AVALIADOR', 'labels']).size().unstack(fill_value=0)

# Normalize to get percentages
sentiment_by_region_percentage = sentiment_by_region.div(sentiment_by_region.sum(axis=1), axis=0)

# Create a 100% stacked bar chart
ax = sentiment_by_region_percentage.plot(kind='bar', stacked=True, figsize=(12, 6))

# Add titles and labels
plt.title('Distribuição de Sentimento por Região (100% Empilhado)', fontsize=16)
plt.xlabel('Região', fontsize=12)
plt.ylabel('Proporção de Reviews', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Sentimento')

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))

# Clean up the plot
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_reviews['REGIAO_AVALIADOR'], df_reviews['labels'])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-Square Test for REGIAO_AVALIADOR vs labels")
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")

# Interpret the results
alpha = 0.05
if p < alpha:
    print("\nConclusion: There is a statistically significant association between region and sentiment (p < 0.05).")
else:
    print("\nConclusion: There is no statistically significant association between region and sentiment (p >= 0.05).")

## Sentimento Presente vs Não-Presente


In [ ]:
# Group by region and sentiment, then count occurrences
sentiment_by_region = df_reviews.groupby(['FLG_PRESENTE', 'labels']).size().unstack(fill_value=0)

# Normalize to get percentages
sentiment_by_region_percentage = sentiment_by_region.div(sentiment_by_region.sum(axis=1), axis=0)

# Create a 100% stacked bar chart
ax = sentiment_by_region_percentage.plot(kind='bar', stacked=True, figsize=(8, 4))

# Add titles and labels
plt.title('Distribuição de Sentimento: Presente vs. Não-Presente', fontsize=16)
plt.xlabel('Tipo de Compra', fontsize=12)
plt.ylabel('Número de Reviews', fontsize=12)
plt.legend(title='Sentimento')
plt.xticks(ticks=[0, 1], labels=['Não-Presente', 'Presente'], rotation=0)

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))

# Clean up the plot
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
contagem_abs = df_reviews.groupby(['COD_PRODUTO', 'FLG_PRESENTE'])['labels'].value_counts().rename('contagem_absoluta')

proporcao = df_reviews.groupby(['COD_PRODUTO', 'FLG_PRESENTE'])['labels'].value_counts(normalize=True).rename('proporcao')

resultado = pd.concat([contagem_abs, proporcao], axis=1)

resultado